## Lab 3: Where is the nearest cafe?

**Objectives:**
   * We will explore OpenStreetMap (OSM) data using <code>osmnx</code>.
   * Learn about OSM data stuctures
   * Compute walking distances using just a few lines of code
   * Visualize our data using `folium`

We will be relying heavily on <code>osmnx</code> which is a package that enables automatic download of geospatial data from OSM and allows us to model, project, visualize, and analyze real-world street networks. More information about this package can be found [here](https://osmnx.readthedocs.io/en/stable/) and the [GitHub repo](https://github.com/gboeing/osmnx/tree/6f9236f20a81416bf34186a811a8ebb76afa0dc8)

We will start with a simple example that calculates the Euclidean distance between Condon Hall and some cafes in Eugene. After that, we will use the <code>networkx</code> package to calculate actual walking distances from Condon Hall to the cafes.

### Create a new environment and launch the notebook

As always, make a new environment that contains all the packages for this lab. Navigate to the `labs/lab3` folder (either from the terminal for Linux and Mac users or from the **CMD.exe Prompt** launched from **Anaconda Navigator** for Windows users) and run:

`conda env create -f environment.yml`

Activate this environment by running:

`conda activate lab3`

Now launch the notebook by running:

`jupyter notebook 03_walking_distances.ipynb`

In [ ]:
# Import modules
import osmnx as ox 

import numpy as np
import geopandas as gpd

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Retrieve OSM data

First we will retrieve all features labelled (or tagged) as buildings in Eugene.

In [ ]:
# Specify type of data
tags = {'building': True}

# Download building geometries from OSM
gdf = ox.geometries_from_place('Eugene, Oregon, USA', tags)

It did not take long to download all this data even though this GeoDataFrame is massive (don't worry about the error message!). We can find the number of buildings in Eugene by printing the shape of the GeoDataFrame.

In [ ]:
print(gdf.shape)

We can find out what attributes are contained in this GeoDataFrame by printing the column headers as a list. 

In [ ]:
print(gdf.columns.tolist())

Since OSM is entirely based on volunteered geographic information, there are a lot of attributes, most of which are not complete. We can find the number of non-NaNs in each column by typing:

In [ ]:
# Count number of non-NaNs in each
warnings.filterwarnings("ignore", category=DeprecationWarning)
gdf.count()

For the purposes of this assignment, we are only interested in buildings that are tagged as **cafes**. Cafes are usually tagged as an **amenity** in OSM so we can filter them using a string comparison.

In [ ]:
# Filter cafes

warnings.filterwarnings("ignore", category=DeprecationWarning)
cafes = gdf[gdf['amenity'] == 'cafe'].reset_index()
cafes

We find that there are 33 cafes in Eugene. This could well be an underestimate since there could be cafes that were not tagged as amenities when mapped. But let's keep moving forward. 

### Computing Euclidean distances to cafes from Condon Hall

We now want to compute the distance to ten nearest cafes. First we will have to reproject our data to UTM Zone 10N which is a projection system that contains Oregon (https://epsg.io/32610)

In [ ]:
# Reproject to UTM Zone 10N
warnings.filterwarnings("ignore", category=DeprecationWarning)
gdf = gdf.to_crs('EPSG:32610')
cafes = cafes.to_crs('EPSG:32610')

Find the coordinates of Condon Hall by searching for **Condon Hall** in the original GeoDataFrame.

In [ ]:
# Get coordinates of Condon Hall
warnings.filterwarnings("ignore", category=DeprecationWarning)
condon_hall = gdf[gdf['name'] == 'Condon Hall'].reset_index()

Compute building centroids so we can compute point-to-point distances.

In [ ]:
# Get cafe and Condon Hall centroids
warnings.filterwarnings("ignore", category=DeprecationWarning)
cafes['centroid'] = cafes['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))

condon_hall['centroid'] = condon_hall['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))

Now we can compute the Euclidean distances between Condon Hall and the cafes

In [ ]:
# Compute distances
warnings.filterwarnings("ignore", category=DeprecationWarning)
condon_hall_x = condon_hall['centroid'].x.values[0]
condon_hall_y = condon_hall['centroid'].y.values[0]
distances = np.sqrt(((condon_hall_x - cafes['centroid'].x.values)**2)
                     + ((condon_hall_y - cafes['centroid'].y.values)**2))

# Add to GeoDataFrame
cafes['euclidean_distance'] = distances

...and print a list of the ten closest cafes to Condon Hall

In [ ]:

warnings.filterwarnings("ignore", category=DeprecationWarning)
print(cafes.nsmallest(10, ['euclidean_distance'])[['name', 'euclidean_distance']])

Now for a bit of magic, let's visualize our data using `folium`

In [ ]:
# Make a new DataFrame containing only the three most relevant columns
warnings.filterwarnings("ignore", category=DeprecationWarning)
nearest_cafes = cafes.nsmallest(10, ['euclidean_distance'])[['name', 'euclidean_distance', 'centroid']]

# Set column geometry
nearest_cafes = nearest_cafes.set_geometry('centroid')

# Convert back to WGS84
nearest_cafes = nearest_cafes.to_crs('EPSG:4326')

# Import package
import folium

# Define center of map (i.e. Condon Hall) and initial zoom level
lat_lon = [44.0451, -123.0781]
m = folium.Map(location=lat_lon, zoom_start=15)

for i in range(0, nearest_cafes.shape[0]):
    my_string = 'name: {}, distance: {}'.format(nearest_cafes.iloc[i]['name'], nearest_cafes.iloc[i]['euclidean_distance'])
    folium.Marker([nearest_cafes.iloc[i]['centroid'].y, nearest_cafes.iloc[i]['centroid'].x],
                 popup=my_string).add_to(m)

# Display map
m

Click on the markers above and a pop-up should appear showing the name of the cafe and Euclidean distance (in meters) from Condon Hall.

## Question 1 (20 points):
Write a script that:

* Computes the Euclidean distance to another **amenity** of your choosing (HINT: use `gdf['amenity'].unique()` to list the different amenities). Feel free to download OSM buildings from another place and choose a different home' location.


* Makes an interactive map showing where your ten nearest amenities are using `folium`. 

### Computing network distances

Euclidean distances often underestimate the distance between two objects, especially when there are obstacles between the two. So we will now compute some more realistic distances to cafes around Condon Hall. First we will need to import the [<code>networkx</code>](https://networkx.org/) package which will allow us conduct a network analysis. 

In [ ]:
# Import module
import networkx as nx

Now we will define the coordinates of Condon Hall and download a walkable street network from OSM. Since the furthest of our ten cafes was 1.5 km away, we will limit our download to 1.6 km.

In [ ]:
# Define coordinates of Condon Hall
warnings.filterwarnings("ignore", category=DeprecationWarning)
lat_lon = (44.0451, -123.0781)

# Import walkable street network data around Condon Hall
g = ox.graph_from_point(lat_lon, dist=1600, network_type='walk')

# Plot map
fig, ax = ox.plot_graph(g, node_size=10)

In [ ]:
# Convert to graph
warnings.filterwarnings("ignore", category=DeprecationWarning)
graph_proj = ox.project_graph(g)

# Get edges and nodes separately
nodes_proj, edges_proj = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)

In [ ]:
# Check projection is UTM Zone 10N
warnings.filterwarnings("ignore", category=DeprecationWarning)
print("Coordinate system:", edges_proj.crs)

# Convert the cafe dataset back to UTM Zone 10N
nearest_cafes = nearest_cafes.to_crs('EPSG:32610')

In [ ]:
# Get x and y coordinates of Condon Hall
orig_xy = (condon_hall['centroid'].y.values[0], condon_hall['centroid'].x.values[0])

# Get x and y coordinates of one of the cafes (the furthest of the ten)
target_xy = (nearest_cafes['centroid'].y.values[-1], nearest_cafes['centroid'].x.values[-1])

In [ ]:
# Find the node in the graph that is closest to the origin point (here, we want to get the node id)
warnings.filterwarnings("ignore", category=DeprecationWarning)
orig_node = ox.distance.nearest_nodes(G=graph_proj, X=orig_xy[1], Y=orig_xy[0], return_dist=False)

# Find the node in the graph that is closest to the target point (here, we want to get the node id)
target_node = ox.distance.nearest_nodes(graph_proj, X=target_xy[1], Y=target_xy[0], return_dist=False)

In [ ]:
# Calculate the shortest path
route = nx.shortest_path(G=graph_proj, source=orig_node, target=target_node, weight='length')

In [ ]:
# Plot the shortest path using folium
warnings.filterwarnings("ignore", category=DeprecationWarning)
m = ox.plot_route_folium(g, route, weight=5)


## Question 2 (20 points): 

Adapt the code above to compute the network distance between two points (either in Eugene or in a city of your choice) and show your results using an interactive map. Write a few sentences about what your map shows.

### Compute network distances between Condon Hall and cafes

In [ ]:
# Get the nodes along the shortest path
warnings.filterwarnings("ignore", category=DeprecationWarning)
route_nodes = nodes_proj.loc[route]

# Create a geometry for the shortest path
route_line = LineString(list(route_nodes['geometry'].values))

# Create a GeoDataFrame
route_geom = gpd.GeoDataFrame([[route_line]], geometry='geometry', crs=edges_proj.crs, columns=['geometry'])

# Print length of route
print('Walking distance to %s = %.1f km' % (nearest_cafes['name'].iloc[-1], route_geom['geometry'].length / 1000))

OK that looks very nice, but what about if we wanted distances to all ten cafes. We need to put this code into a for loop.

In [ ]:
# Get x and y coordinates of all ten of the nearest cafes
warnings.filterwarnings("ignore", category=DeprecationWarning)
target_xy = (nearest_cafes['centroid'].y.values, nearest_cafes['centroid'].x.values)

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
routes = []
distances = []
for i in range(len(target_xy[0])):
    
    # Find the node in the graph that is closest to the target point (here, we want to get the node id)
    target_node = ox.distance.nearest_nodes(graph_proj, X=target_xy[1][i], Y=target_xy[0][i], return_dist=False)
    
    # Calculate the shortest path
    route = nx.shortest_path(G=graph_proj, source=orig_node, target=target_node, weight='length')
    
    # Append route to list
    routes.append(route)
    
    # Get the nodes along the shortest path
    route_nodes = nodes_proj.loc[route]
    
    # Create a geometry for the shortest path
    route_line = LineString(list(route_nodes['geometry'].values))

    # Create a GeoDataFrame
    route_geom = gpd.GeoDataFrame([[route_line]], geometry='geometry', crs=edges_proj.crs, columns=['geometry'])

    # Print length of route
    print('Walking distance to %s = %.1f km' % (nearest_cafes['name'].iloc[i], route_geom['geometry'].length / 1000))
    
    # Append distances to list
    distances.append(route_geom['geometry'].length[0])

Let's add the network distances to the original DataFrame and display

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
nearest_cafes['network_distance'] = distances
nearest_cafes

## Question 3 (10 points): 


* a) Calculate the average difference between the Euclidean and network distances for you amenities 


* b) Describe some situations where it would not be advisable to use Euclidean distances?